## import libraries

In [1]:
import torch
import gym

In [2]:
env = gym.make("FrozenLake-v0")
gamma = 0.99
threshold = 0.0001

In [3]:
def policy_evaluation(env, policy, gamma, threshold):
  n_state = policy.shape[0]
  V = torch.zeros(n_state)
  while True:
    V_temp = torch.zeros(n_state)
    for state in range(n_state):
      action = policy[state].item()
      for trans_prob, new_state, reward, _ in env.env.P[state][action]:
        V_temp[state] += trans_prob * (reward + gamma * V[new_state])
    max_delta = torch.max(torch.abs(V - V_temp))
    V = V_temp.clone()
    if max_delta <= threshold:
      break
  return V

In [4]:
def policy_improvement(env, V, gamma):

  n_state = env.observation_space.n
  n_action = env.action_space.n  
  policy = torch.zeros(n_state)
  for state in range(n_state):  
    v_actions = torch.zeros(n_action)
    for action in range(n_action):
      for trans_prob, new_state, reward, _ in env.env.P[state][action]:
        v_actions[action] += trans_prob * (reward + gamma * V[new_state])
    policy[state] = torch.argmax(v_actions)
  return policy

In [5]:
def policy_iteration(env, gamma, threshold):

  n_state = env.observation_space.n
  n_action = env.action_space.n
  policy = torch.randint(high=n_action,size=(n_state,)).float()
  while True:
    V = policy_evaluation(env, policy, gamma, threshold)
    policy_improved = policy_improvement(env, V, gamma)
    if torch.equal(policy_improved, policy):
      return V, policy_improved
    policy = policy_improved

In [6]:
def run_episode(env, policy):
  state = env.reset()
  total_reward = 0
  is_done = False
  while not is_done:
    action = policy[state].item()
    state, reward, is_done, info = env.step(action)
    total_reward += reward
    if is_done:
      break
  return total_reward

In [7]:
V_optimal, optimal_policy = policy_iteration(env, gamma, threshold)

In [8]:
n_episode = 1000
total_rewards = []
for episode in range(n_episode):
  total_reward = run_episode(env, optimal_policy)
  total_rewards.append(total_reward)
print('Average total reward under the optimal policy: {}'.format(sum(total_rewards) / n_episode))

Average total reward under the optimal policy: 0.753
